# Cleanup
Konvertering af brevene i json-format til en enkelt csv-fil.

Vi renser ud i de mest basale tekstformateringer, såsom linjeskift, tabuleringer og dobbelt mellemrum.

Omdøber kolonnerne og konverterer datoerne til datetime-format og introducerer et id.


In [ ]:
# load jsonfile from disk
import json
import pandas as pd


In [ ]:

with open('data/letters.json', 'r') as f:
    letters = json.load(f)

from MLStripper import strip_tags

# strip html tags from text
for letter in letters:
    letter['Text'] = strip_tags(letter['Text'])

df = pd.DataFrame(letters)

# lowercase all column names
df.columns = map(str.lower, df.columns)


In [ ]:
# Text cleanup

# replace crlf with lf
df['text'] = df['text'].str.replace('\r\n', '\n')
# replace leading and trailing whitespace
df['text'] = df['text'].str.strip()

# replace single newlines with space
df['text'] = df['text'].str.replace('\n', ' ')

# replace multiple newlines with single newline
df['text'] = df['text'].str.replace('\n+', '\n')

# replace \xa0 with space
df['text'] = df['text'].str.replace('\xa0', ' ')

# replace \xa with ''
df['text'] = df['text'].str.replace('\xad', '')
df['text'] = df['text'].str.replace('\x95', '')

# replace multiple spaces with single space
df['text'] = df['text'].str.replace('\s+', ' ')


In [ ]:
# Dates
df['date'] = pd.to_datetime(df['letterdate'])
df['date'] = df['date'].dt.date
df.drop(['letterdate'], axis=1, inplace=True)
df.sort_values(by=['date'], inplace=True)


In [ ]:
# Add an id column as the first column
df.insert(0, 'id', range(1, 1 + len(df)))

# reorder columns
df = df[['id', 'date', 'sender', 'recipient','place','location', 'text']]

df.to_csv('data/letters.csv', index=False)


In [ ]:
# ten random letters from the dataset, sender "Peter Mærsk"
randomletters = df[df['sender'] == 'Peter Mærsk'].sample(10, random_state=1).copy()
# order by id asc
randomletters.sort_values(by=['id'], inplace=True)
randomletters['text'] =randomletters['text'].str.slice(0, 500)+' [...]'
randomletters.to_markdown('data/ten_random_letters.md', index=False)

